In [1]:
import os

# change the working directory to root of this project
os.chdir("..")

# verify the working directory
os.getcwd()

'/home/abraham-pc/Documents/personal_projects/Multi-Platform_Job_Recommender/backend'

In [2]:
import os
import dotenv
import yaml
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

In [3]:
# load environment variables from .env file
dotenv.load_dotenv(dotenv_path="./config/.env")

# load configuration from config.yaml
with open("./config/config.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [4]:
username = config["database"]["cassandra"]["username"]
password = os.getenv('CASSANDRA_PASSWORD')
cluster = Cluster(port=29042, auth_provider=PlainTextAuthProvider(username=username, password=password))

In [5]:
session = cluster.connect()

In [7]:
sample = session.execute("select release_version from system.local")

In [8]:
for i in sample:
    print(i)

Row(release_version='4.1.3')


In [21]:
from backend.etl.databases.cassandra.cassandra_conn import CassandraConn
from backend.etl.databases.cassandra.table_models import JobListings

conn = CassandraConn()
session = conn.session


In [3]:
uuids_set = session.execute(
            "SELECT uuid FROM job_listings"
        )
uuids = []
for i in uuids_set:
    uuids.append(str(i['uuid']))

In [4]:
scrape_dates_set = session.execute(
    "SELECT scraped_at FROM job_listings"
)
scrape_dates = []
for i in scrape_dates_set:
    scrape_dates.append(i['scraped_at'])

In [6]:
from datetime import datetime

In [12]:
today = datetime.today() - datetime(2023, 10, 27)
today.days

31

In [14]:
jobs_set = session.execute(
    "SELECT * FROM job_listings LIMIT 5"
)
jobs = []
for job in jobs_set:
    jobs.append(job)

In [15]:
jobs

[{'uuid': UUID('5c1b0dff-cafb-fce0-5088-5d42776652e8'),
  'company_name': 'Sundry Foods Limited',
  'date': '2023-10-13',
  'emp_type': '\n            Full-time\n          ',
  'ind': '\n            Hospitality\n            ',
  'job_desc': "\nNever Miss a Job Update Again. Company: Location: NigeriaState: Job type: Full-Time Job category: Job DescriptionSundry Foods is an integrated food services company founded in 2003 and operating in major cities in Nigeria. We are bringing our now trademark great tasting ready-to-eat food and quality services to thousands of people and institutions daily through its chain and network of restaurants, bakeries and other catering facilities.We are proud to own and operate one of Africa’s best restaurant and bakery chains and we pride ourselves on our ability to consistently deliver original food and service solutions whether in workplaces, schools and colleges, hospitals, joining even remote sites and different cultural environments using our team of

In [19]:
full_text = str()

cols = list(jobs[0].keys())[1:]

for col in cols:
    full_text += str(jobs[0][col]) + " "

In [20]:
full_text

"Sundry Foods Limited 2023-10-13 \n            Full-time\n           \n            Hospitality\n             \nNever Miss a Job Update Again. Company: Location: NigeriaState: Job type: Full-Time Job category: Job DescriptionSundry Foods is an integrated food services company founded in 2003 and operating in major cities in Nigeria. We are bringing our now trademark great tasting ready-to-eat food and quality services to thousands of people and institutions daily through its chain and network of restaurants, bakeries and other catering facilities.We are proud to own and operate one of Africa’s best restaurant and bakery chains and we pride ourselves on our ability to consistently deliver original food and service solutions whether in workplaces, schools and colleges, hospitals, joining even remote sites and different cultural environments using our team of young seasoned professionals and an entire workforce with a passion to deliver nothing but the best to every customer.We are recruit

In [23]:
job = JobListings.objects(uuid=uuids[0]).get()
dict(job)

{'uuid': UUID('5c1b0dff-cafb-fce0-5088-5d42776652e8'),
 'skipped': False,
 'scraped_at': datetime.datetime(2023, 11, 27, 18, 44, 44, 651000),
 'source': 'Linkedin',
 'job_id': 'None',
 'job_title': 'Restaurant Management Trainee at Sundry Foods Limited (11 Openings)',
 'company_name': 'Sundry Foods Limited',
 'location': 'Nigeria',
 'date': '2023-10-13',
 'job_link': 'https://ng.linkedin.com/jobs/view/restaurant-management-trainee-at-sundry-foods-limited-11-openings-at-sundry-foods-limited-3734968720?refId=kwKLKGAoUFs%2Fvff96FXfjg%3D%3D&trackingId=xVJLYIyzF8EyFXoYT85rtg%3D%3D&position=16&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'job_desc': "\nNever Miss a Job Update Again. Company: Location: NigeriaState: Job type: Full-Time Job category: Job DescriptionSundry Foods is an integrated food services company founded in 2003 and operating in major cities in Nigeria. We are bringing our now trademark great tasting ready-to-eat food and quality services to thousands of people and